In [30]:
import tensorflow as tf
AUTOTUNE = tf.compat.v1.estimator.inputs

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [31]:
import zipfile
with zipfile.ZipFile('./faces_TM.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [32]:
test_persons = ['sz24', 'megak', 'night', 'choon', 'kawamura']

In [33]:
from PIL import Image
import glob
image_list = []
for filepath in glob.glob('faces_TM/*/*.jpg', recursive=True):
    filename = filepath.split("/")[-1]
    person = filepath.split("/")[1]
    label = filename.split("_")[1]
    test_train = person in test_persons
    image_list.append((filepath, label, int(test_train)))
    
# Create a data frame
data = pd.DataFrame(data=image_list, columns=['image_path', 'label', 'isTest'])

# Convert string labels to numeric
d = {'left':0, 'right':1, 'straight':2, 'up':3}
data['labels_num'] = data['label'].map(d, na_action='ignore')

# Generate two data frames for training and validation
train_df = data[data['isTest']==0]
validation_df = data[data['isTest']==1]
print('Train size: {}, Test size: {}'.format(train_df.shape[0], validation_df.shape[0] ) )
N_train_images = train_df.shape[0]
N_val_images = validation_df.shape[0]

Train size: 469, Test size: 155


In [34]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input, Lambda
from tensorflow.keras import regularizers, optimizers

# Input layer
input_ = Input(shape=(28, 28, 3)) # This is the input shape
# Keras data loader does not like to read 1 channel images. This Lambda finction removes the extra channels added by data loader.
input_slice = Lambda(lambda x: tf.expand_dims(x[:,:,:,0], -1, name=None))(input_)
x = Flatten()(input_slice) # This will convert the 28x28 input to a vector of dimension 784

# Hidden layer
h = Dense(64)(x)
h = Activation('sigmoid')(h)

# Output layer
out_ = Dense(4)(h)
out_ = Activation('softmax')(out_)
model = Model(inputs=input_, outputs=out_) # Setup the model

In [35]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
                optimizer=sgd,
                metrics=['accuracy'])

In [36]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28, 28, 3)]       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                50240     
_________________________________________________________________
activation_4 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 260       
_________________________________________________________________
activation_5 (Activation)    (None, 4)                 0   

In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
batch_size = 16
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='./',
        x_col="image_path",
        y_col="label",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')
validation_generator = val_datagen.flow_from_dataframe(
        dataframe=validation_df,
        directory='./',
        x_col="image_path",
        y_col="label",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

Found 469 validated image filenames belonging to 4 classes.
Found 155 validated image filenames belonging to 4 classes.


In [38]:
def calculate_losses(model_, data_generator_, N_images, batch_size_):
    loss_hold = []
    acc_hold = []
    batches = 0
    for x,y in data_generator_:
        loss,acc = model_.evaluate(x, y, verbose=0)
        loss_hold.append(loss)
        acc_hold.append(acc)
        batches += 1
        if batches >= N_images / batch_size_:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    return np.mean(loss_hold), np.mean(acc_hold)

In [39]:
res = []
for e in range(50): # Train for 50 epoch
    print('Epoch', e)
    batches = 0
    
    loss_ = []
    acc_ = []
    
    # iterate over each batch
    for x,y in train_generator:
        loss, acc = model.train_on_batch(x, y)
        loss_.append(loss)
        acc_.append(acc)
        batches += 1
        if batches >= N_train_images / batch_size:
            break
    loss_ = np.mean(loss_)
    acc_ = np.mean(acc_)
    print("Training epoch {}: Loss = {}, Accuracy = {}".format(e, loss_, acc_))
    
    loss, acc = calculate_losses(model, validation_generator, N_val_images, batch_size)
    print("Validation epoch {}: Loss = {}, Accuracy = {}".format(e, loss, acc))
    
    res.append((e, loss_, acc_, loss, acc))

Epoch 0
Training epoch 0: Loss = 1.3950610160827637, Accuracy = 0.2929166853427887
Validation epoch 0: Loss = 1.2879704356193542, Accuracy = 0.42045456171035767
Epoch 1
Training epoch 1: Loss = 1.215722918510437, Accuracy = 0.6002469658851624
Validation epoch 1: Loss = 1.0837776899337768, Accuracy = 0.6789773106575012
Epoch 2
Training epoch 2: Loss = 1.0805022716522217, Accuracy = 0.6551080942153931
Validation epoch 2: Loss = 0.9433213829994201, Accuracy = 0.6119318008422852
Epoch 3
Training epoch 3: Loss = 0.9577205777168274, Accuracy = 0.7056789994239807
Validation epoch 3: Loss = 0.8281258940696716, Accuracy = 0.7630681991577148
Epoch 4
Training epoch 4: Loss = 0.8503329157829285, Accuracy = 0.7701080441474915
Validation epoch 4: Loss = 0.7202162265777587, Accuracy = 0.7602273225784302
Epoch 5
Training epoch 5: Loss = 0.7601014971733093, Accuracy = 0.7772067785263062
Validation epoch 5: Loss = 0.6719814896583557, Accuracy = 0.8596590757369995
Epoch 6
Training epoch 6: Loss = 0.69965